In [1]:
import os
from birdsong.config import config
from birdsong.audiotransform.to_image import AudioPreprocessor


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from tensorflow.keras import models
from tensorflow.keras.models import Sequential

from tensorflow.keras.backend import expand_dims
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard


**Preparing the data**

In [3]:
data_input_path = '../data/split_data/'
data_output_path = '../data/model_data/'

df = AudioPreprocessor(data_input_path,data_output_path,config.SPECTOGRAM_TYPE, config.OUTPUT_FORMAT)


In [ ]:
# create train dataset

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_output_path,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(64, 376),
  batch_size=32,
  color_mode = "grayscale")


In [ ]:
# create validation dataset

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_output_path,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(64, 376),
  batch_size=32,
  color_mode = "grayscale")


In [ ]:
print(f"train dataset shape: {train_ds.shape}")
print(f"validation dataset shape: {val_ds.shape}")


In [ ]:
# print class labels of train dataset
class_labels = train_ds.class_names
print(class_labels)
nb_classes = len(class_labels)
print(f"There are {nb_classes} target labels")


Model architecture : MFF-ScSEnet 

Layers:
-Rescaling layer
...

Add:
-Batch normalization
-Dropout layer



In [ ]:
def initialize_model():

    model = models.Sequential()

    model.add(layers.Rescaling(1./255), input_shape=())

    model.add(layers.Conv2D(8, (4,4), padding='same', activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))


    model.add(layers.Conv2D(16, (3,3), activation="relu", padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))


    model.add(layers.Flatten())

    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(10, activation='relu')) # intermediate layer

    ### Last layer - Classification Layer
    model.add(layers.Dense(nb_classes, activation='softmax'))

    ### Model compilation
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']
    )
    return model


In [ ]:
# print number of trainable parameters

model = initialize_model()
model.summary()


In [ ]:
# set early stopping and model checkpoint params

es = EarlyStopping(restore_best_weights=True, patience=3, monitor="val_accuracy")

checks = ModelCheckpoint(os.path.join('..','data','models','checkpoints'),
                         monitor='val_loss', save_weights_only=False,
                         save_freq='epoch', mode='min', save_best_only=True)


In [ ]:
# train the model

history = model.fit(train_ds,
          validation_data=val_ds,
          batch_size=32,
          epochs=5,
          callbacks=[es, checks],
          verbose=1)
